In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

from scipy import stats as sps
from scipy.interpolate import interp1d

from IPython.display import clear_output

In [2]:
import copy, math

In [3]:
import plotly.express as px

In [4]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
#import plotly.graph_objects as go
init_notebook_mode(connected=True)

In [6]:
def format_comma(num):
    return f'{num:,.0f}'

# Setup

## Define all keys

In [7]:
state_key = {
'EC':'Eastern Cape',
'FS':'Free State',
'GP':'Gauteng',
'KZN':'Kwazulu Natal',
'LP':'Limpopo',
'MP':'Mpumalanga',
'NC':'Northern Cape',
'NW':'North-West',
'WC':'Western Cape'
}
state_filter = list(state_key.keys())
state_filter

['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

In [8]:
state_labels = list(state_key.values())
state_labels

['Eastern Cape',
 'Free State',
 'Gauteng',
 'Kwazulu Natal',
 'Limpopo',
 'Mpumalanga',
 'Northern Cape',
 'North-West',
 'Western Cape']

In [9]:
#not required
state_filter_d = copy.deepcopy(state_filter)
state_filter_d.append('Date')
state_filter_d

['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC', 'Date']

In [10]:
state_filter_t = copy.deepcopy(state_filter)
state_filter_t.insert(0,'Total RSA')
state_filter_t

['Total RSA', 'EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

In [11]:
state_filter_all = copy.deepcopy(state_filter)
state_filter_all.insert(0,'Total RSA')
state_filter_all.append('Date')
state_filter_all

['Total RSA', 'EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC', 'Date']

## Download SA population stats

In [12]:
# Mid-year 2019 Estimates
# https://www.statssa.gov.za/publications/P0302/P03022019.pdf
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/district_data/za_province_pop.csv'
province_pops = pd.read_csv(url, header=None, names=['Province','Pop'])
province_pops

,Province,Pop
0,Gauteng,15176115
1,KwaZulu-Natal,11289086
2,Western Cape,6844272
3,Eastern Cape,6712276
4,Limpopo,5982584
5,Mpumalanga,4592187
6,Northwest,4027160
7,Free State,2887465
8,Northern Cape,1263875


In [13]:
country_pop = province_pops['Pop'].sum()
country_pop

58775020

In [14]:
province_pops.iloc[0]['Pop']

15176115

In [15]:
state_pop = {
    'EC': 6712276,
    'FS': 2887465,
    'GP': 15176115,
    'KZN': 11289086,
    'LP': 5982584,
    'MP': 4592187,
    'NC': 1263875,
    'NW': 4027160,
    'WC': 6844272
}

# Download Data

## Download case data

In [16]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv'
states_cases_i = pd.read_csv(url, parse_dates=['date'], dayfirst=True, squeeze=True, index_col=0)
states_cases_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-03-05,20200305,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
2020-03-07,20200307,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,NaN
2020-03-08,20200308,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3,NaN
2020-03-09,20200309,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7,NaN
2020-03-11,20200311,0.0,0.0,5.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903,86489.0,38869.0,211678.0,114034.0,13488.0,24746.0,11313.0,25816.0,106582.0,0.0,633015,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904,86635.0,39192.0,212186.0,114360.0,13596.0,24836.0,11521.0,25965.0,106787.0,0.0,635078,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905,86755.0,39486.0,212564.0,114646.0,13665.0,24976.0,11718.0,26092.0,106982.0,0.0,636884,https://sacoronavirus.co.za/2020/09/05/update-...


In [17]:
casezero = states_cases_i.index[0]
caselast = states_cases_i.index[-1]
casezero, caselast

(Timestamp('2020-03-05 00:00:00'), Timestamp('2020-09-07 00:00:00'))

In [18]:
idx = pd.date_range(casezero, caselast)
idx

DatetimeIndex(['2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14',
               ...
               '2020-08-29', '2020-08-30', '2020-08-31', '2020-09-01',
               '2020-09-02', '2020-09-03', '2020-09-04', '2020-09-05',
               '2020-09-06', '2020-09-07'],
              dtype='datetime64[ns]', length=187, freq='D')

In [19]:
states_cases_i = states_cases_i.reindex(idx, method='ffill')
states_cases_i = states_cases_i.rename(columns={'total':'Total RSA'})
states_cases_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,Total RSA,source
2020-03-05,20200305,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
2020-03-06,20200305,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
2020-03-07,20200307,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,NaN
2020-03-08,20200308,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3,NaN
2020-03-09,20200309,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903,86489.0,38869.0,211678.0,114034.0,13488.0,24746.0,11313.0,25816.0,106582.0,0.0,633015,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904,86635.0,39192.0,212186.0,114360.0,13596.0,24836.0,11521.0,25965.0,106787.0,0.0,635078,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905,86755.0,39486.0,212564.0,114646.0,13665.0,24976.0,11718.0,26092.0,106982.0,0.0,636884,https://sacoronavirus.co.za/2020/09/05/update-...
2020-09-06,20200906,86849.0,39781.0,212898.0,114824.0,13734.0,25076.0,11923.0,26245.0,107187.0,0.0,638517,https://twitter.com/nicd_sa/status/13026906284...


In [20]:
states_cases = states_cases_i.copy()
states_cases = states_cases.reset_index()
states_cases = states_cases.rename(columns={'index':'Date'})
states_cases

,Date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,Total RSA,source
0,2020-03-05,20200305,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
1,2020-03-06,20200305,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
2,2020-03-07,20200307,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,NaN
3,2020-03-08,20200308,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3,NaN
4,2020-03-09,20200309,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2020-09-03,20200903,86489.0,38869.0,211678.0,114034.0,13488.0,24746.0,11313.0,25816.0,106582.0,0.0,633015,https://twitter.com/nicd_sa/status/13016021023...
183,2020-09-04,20200904,86635.0,39192.0,212186.0,114360.0,13596.0,24836.0,11521.0,25965.0,106787.0,0.0,635078,https://twitter.com/nicd_sa/status/13019655662...
184,2020-09-05,20200905,86755.0,39486.0,212564.0,114646.0,13665.0,24976.0,11718.0,26092.0,106982.0,0.0,636884,https://sacoronavirus.co.za/2020/09/05/update-...
185,2020-09-06,20200906,86849.0,39781.0,212898.0,114824.0,13734.0,25076.0,11923.0,26245.0,107187.0,0.0,638517,https://twitter.com/nicd_sa/status/13026906284...


## Download death data

In [21]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_deaths.csv'
states_deaths_i = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True,index_col=0).sort_index()
states_deaths_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-03-27,20200327,0,0,0,0,0,0,0,0,1,0,1,https://sacoronavirus.co.za/2020/03/27/latest-...
2020-03-28,20200328,0,0,0,1,0,0,0,0,1,0,2,NaN
2020-03-30,20200330,0,1,0,1,0,0,0,0,1,0,3,NaN
2020-03-31,20200331,0,1,1,2,0,0,0,0,1,0,5,NaN
2020-04-03,20200403,0,1,1,6,0,0,0,0,1,0,9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903,2956,739,3730,2203,202,378,134,275,3946,0,14563,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904,2968,757,3773,2218,214,378,145,275,3950,0,14678,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905,2979,776,3789,2263,214,378,145,275,3960,0,14779,https://twitter.com/nicd_sa/status/13023151403...


In [22]:
states_deaths_i = states_deaths_i.reindex(idx, method='ffill')
states_deaths_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903.0,2956.0,739.0,3730.0,2203.0,202.0,378.0,134.0,275.0,3946.0,0.0,14563.0,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904.0,2968.0,757.0,3773.0,2218.0,214.0,378.0,145.0,275.0,3950.0,0.0,14678.0,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905.0,2979.0,776.0,3789.0,2263.0,214.0,378.0,145.0,275.0,3960.0,0.0,14779.0,https://twitter.com/nicd_sa/status/13023151403...
2020-09-06,20200906.0,2986.0,785.0,3813.0,2294.0,214.0,416.0,145.0,275.0,3961.0,0.0,14889.0,https://twitter.com/nicd_sa/status/13026906284...


In [23]:
states_deaths_i.iloc[0, :] = states_deaths_i.iloc[0, :].replace({np.nan:0})
states_deaths_i = states_deaths_i.ffill(axis=0)
states_deaths_i = states_deaths_i.rename(columns={'total':'Total RSA'})
states_deaths_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,Total RSA,source
2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903.0,2956.0,739.0,3730.0,2203.0,202.0,378.0,134.0,275.0,3946.0,0.0,14563.0,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904.0,2968.0,757.0,3773.0,2218.0,214.0,378.0,145.0,275.0,3950.0,0.0,14678.0,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905.0,2979.0,776.0,3789.0,2263.0,214.0,378.0,145.0,275.0,3960.0,0.0,14779.0,https://twitter.com/nicd_sa/status/13023151403...
2020-09-06,20200906.0,2986.0,785.0,3813.0,2294.0,214.0,416.0,145.0,275.0,3961.0,0.0,14889.0,https://twitter.com/nicd_sa/status/13026906284...


In [24]:
states_deaths = states_deaths_i.copy()
states_deaths = states_deaths.reset_index()
states_deaths = states_deaths.rename(columns={'index':'Date'})
states_deaths

,Date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,Total RSA,source
0,2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2020-09-03,20200903.0,2956.0,739.0,3730.0,2203.0,202.0,378.0,134.0,275.0,3946.0,0.0,14563.0,https://twitter.com/nicd_sa/status/13016021023...
183,2020-09-04,20200904.0,2968.0,757.0,3773.0,2218.0,214.0,378.0,145.0,275.0,3950.0,0.0,14678.0,https://twitter.com/nicd_sa/status/13019655662...
184,2020-09-05,20200905.0,2979.0,776.0,3789.0,2263.0,214.0,378.0,145.0,275.0,3960.0,0.0,14779.0,https://twitter.com/nicd_sa/status/13023151403...
185,2020-09-06,20200906.0,2986.0,785.0,3813.0,2294.0,214.0,416.0,145.0,275.0,3961.0,0.0,14889.0,https://twitter.com/nicd_sa/status/13026906284...


## Download recovery data

In [25]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_recoveries.csv'
states_recovery_i = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True,index_col=0).sort_index()
states_recovery_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
date,,,,,,,,,,,,,
2020-04-17,20200417,9,71,479,96,18,9,10,6,205,0,903,https://twitter.com/nicd_sa/status/12512151947...
2020-04-18,20200418,9,71,479,96,18,9,10,6,205,0,903,https://twitter.com/nicd_sa/status/12515952235...
2020-04-19,20200419,9,71,479,96,18,9,10,6,205,0,903,https://twitter.com/nicd_sa/status/12519270552...
2020-04-20,20200420,15,74,545,151,21,14,13,6,216,0,1055,https://twitter.com/nicd_sa/status/12523098244...
2020-04-21,20200421,15,74,545,151,21,14,13,6,216,0,1055,https://twitter.com/nicd_sa/status/12526435556...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903,82233,26056,184538,100766,12647,23046,7526,20028,98047,0,554887,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904,82233,27129,185192,101034,12647,23315,7551,20028,98689,0,557818,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905,82233,27129,186731,101797,12702,23475,7573,20554,99010,0,561204,https://twitter.com/nicd_sa/status/13023151403...


In [26]:
states_recovery_i = states_recovery_i.reindex(idx, method='ffill')
states_recovery_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,total,source
2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903.0,82233.0,26056.0,184538.0,100766.0,12647.0,23046.0,7526.0,20028.0,98047.0,0.0,554887.0,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904.0,82233.0,27129.0,185192.0,101034.0,12647.0,23315.0,7551.0,20028.0,98689.0,0.0,557818.0,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905.0,82233.0,27129.0,186731.0,101797.0,12702.0,23475.0,7573.0,20554.0,99010.0,0.0,561204.0,https://twitter.com/nicd_sa/status/13023151403...
2020-09-06,20200906.0,82233.0,27129.0,187301.0,101989.0,12782.0,23610.0,8717.0,21033.0,99097.0,0.0,563891.0,https://twitter.com/nicd_sa/status/13026906284...


In [27]:
states_recovery_i.iloc[0, :] = states_recovery_i.iloc[0, :].replace({np.nan:0})
states_recovery_i = states_recovery_i.ffill(axis=0)
states_recovery_i = states_recovery_i.rename(columns={'total':'Total RSA'})
states_recovery_i

,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,Total RSA,source
2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-03,20200903.0,82233.0,26056.0,184538.0,100766.0,12647.0,23046.0,7526.0,20028.0,98047.0,0.0,554887.0,https://twitter.com/nicd_sa/status/13016021023...
2020-09-04,20200904.0,82233.0,27129.0,185192.0,101034.0,12647.0,23315.0,7551.0,20028.0,98689.0,0.0,557818.0,https://twitter.com/nicd_sa/status/13019655662...
2020-09-05,20200905.0,82233.0,27129.0,186731.0,101797.0,12702.0,23475.0,7573.0,20554.0,99010.0,0.0,561204.0,https://twitter.com/nicd_sa/status/13023151403...
2020-09-06,20200906.0,82233.0,27129.0,187301.0,101989.0,12782.0,23610.0,8717.0,21033.0,99097.0,0.0,563891.0,https://twitter.com/nicd_sa/status/13026906284...


In [28]:
states_recovery = states_recovery_i.copy()
states_recovery = states_recovery.reset_index()
states_recovery = states_recovery.rename(columns={'index':'Date'})
states_recovery

,Date,YYYYMMDD,EC,FS,GP,KZN,LP,MP,NC,NW,WC,UNKNOWN,Total RSA,source
0,2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,2020-09-03,20200903.0,82233.0,26056.0,184538.0,100766.0,12647.0,23046.0,7526.0,20028.0,98047.0,0.0,554887.0,https://twitter.com/nicd_sa/status/13016021023...
183,2020-09-04,20200904.0,82233.0,27129.0,185192.0,101034.0,12647.0,23315.0,7551.0,20028.0,98689.0,0.0,557818.0,https://twitter.com/nicd_sa/status/13019655662...
184,2020-09-05,20200905.0,82233.0,27129.0,186731.0,101797.0,12702.0,23475.0,7573.0,20554.0,99010.0,0.0,561204.0,https://twitter.com/nicd_sa/status/13023151403...
185,2020-09-06,20200906.0,82233.0,27129.0,187301.0,101989.0,12782.0,23610.0,8717.0,21033.0,99097.0,0.0,563891.0,https://twitter.com/nicd_sa/status/13026906284...


## Download testing data
Only for country

In [29]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_testing.csv'
states_tests_i = pd.read_csv(url, parse_dates=['date'], dayfirst=True, index_col=0)
states_tests_i = states_tests_i['cumulative_tests']
states_tests_i

date
2020-02-11         61
2020-02-13         67
2020-02-14         71
2020-02-19         95
2020-02-20        106
               ...   
2020-09-03    3747101
2020-09-04    3765700
2020-09-05    3783823
2020-09-06    3800190
2020-09-07    3808 49
Name: cumulative_tests, Length: 192, dtype: object

In [30]:
states_tests_i = states_tests_i.reindex(idx, method='ffill')
states_tests_i

2020-03-05        164
2020-03-06        200
2020-03-07        241
2020-03-08        241
2020-03-09        241
               ...   
2020-09-03    3747101
2020-09-04    3765700
2020-09-05    3783823
2020-09-06    3800190
2020-09-07    3808 49
Freq: D, Name: cumulative_tests, Length: 187, dtype: object

In [31]:
states_tests_i = states_tests_i.ffill(axis=0)
states_tests_i = states_tests_i.rename('Total RSA')
states_tests_i

2020-03-05        164
2020-03-06        200
2020-03-07        241
2020-03-08        241
2020-03-09        241
               ...   
2020-09-03    3747101
2020-09-04    3765700
2020-09-05    3783823
2020-09-06    3800190
2020-09-07    3808 49
Freq: D, Name: Total RSA, Length: 187, dtype: object

In [32]:
states_tests = states_tests_i.copy()
states_tests = states_tests.reset_index()
states_tests = states_tests.rename(columns={'index':'Date'})
states_tests

,Date,Total RSA
0,2020-03-05,164
1,2020-03-06,200
2,2020-03-07,241
3,2020-03-08,241
4,2020-03-09,241
...,...,...
182,2020-09-03,3747101
183,2020-09-04,3765700
184,2020-09-05,3783823
185,2020-09-06,3800190


# Analysis Per Province

In [33]:
#https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express
colour_series = px.colors.qualitative.Vivid

In [34]:
filter_cases = states_cases[state_filter_all]
analysis_cases = filter_cases.melt(id_vars='Date', var_name='Province', value_name='Value')
analysis_cases['Data'] = 'Cases'

filter_recovery = states_recovery[state_filter_all]
analysis_recovery = filter_recovery.melt(id_vars='Date', var_name='Province', value_name='Value')
analysis_recovery['Data'] = 'Recovered'

filter_deaths = states_deaths[state_filter_all]
analysis_deaths = filter_deaths.melt(id_vars='Date', var_name='Province', value_name='Value')
analysis_deaths['Data'] = 'Deaths'

In [35]:
filter_add = pd.concat([filter_deaths, filter_recovery])
filter_add = filter_add.groupby('Date').sum()
filter_add

,Total RSA,EC,FS,GP,KZN,LP,MP,NC,NW,WC
Date,,,,,,,,,,
2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-09-03,569450.0,85189.0,26795.0,188268.0,102969.0,12849.0,23424.0,7660.0,20303.0,101993.0
2020-09-04,572496.0,85201.0,27886.0,188965.0,103252.0,12861.0,23693.0,7696.0,20303.0,102639.0
2020-09-05,575983.0,85212.0,27905.0,190520.0,104060.0,12916.0,23853.0,7718.0,20829.0,102970.0


In [36]:
filter_sub = filter_add.rmul(-1).reset_index()
filter_sub

,Date,Total RSA,EC,FS,GP,KZN,LP,MP,NC,NW,WC
0,2020-03-05,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
1,2020-03-06,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2,2020-03-07,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
3,2020-03-08,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
4,2020-03-09,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...
182,2020-09-03,-569450.0,-85189.0,-26795.0,-188268.0,-102969.0,-12849.0,-23424.0,-7660.0,-20303.0,-101993.0
183,2020-09-04,-572496.0,-85201.0,-27886.0,-188965.0,-103252.0,-12861.0,-23693.0,-7696.0,-20303.0,-102639.0
184,2020-09-05,-575983.0,-85212.0,-27905.0,-190520.0,-104060.0,-12916.0,-23853.0,-7718.0,-20829.0,-102970.0
185,2020-09-06,-578780.0,-85219.0,-27914.0,-191114.0,-104283.0,-12996.0,-24026.0,-8862.0,-21308.0,-103058.0


In [37]:
filter_active_i = pd.concat([filter_cases, filter_sub])
filter_active_i = filter_active_i.groupby('Date').sum()
filter_active = filter_active_i.reset_index()
filter_active = filter_active.rename(columns={'index':'Date'})
filter_active

,Date,Total RSA,EC,FS,GP,KZN,LP,MP,NC,NW,WC
0,2020-03-05,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-06,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-07,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-08,3.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
4,2020-03-09,7.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
182,2020-09-03,63565.0,1300.0,12074.0,23410.0,11065.0,639.0,1322.0,3653.0,5513.0,4589.0
183,2020-09-04,62582.0,1434.0,11306.0,23221.0,11108.0,735.0,1143.0,3825.0,5662.0,4148.0
184,2020-09-05,60901.0,1543.0,11581.0,22044.0,10586.0,749.0,1123.0,4000.0,5263.0,4012.0
185,2020-09-06,59737.0,1630.0,11867.0,21784.0,10541.0,738.0,1050.0,3061.0,4937.0,4129.0


In [38]:
analysis_active = filter_active.melt(id_vars='Date', var_name='Province', value_name='Value')
analysis_active['Data'] = 'Active'

In [39]:
analysis_all = pd.concat([analysis_cases, analysis_active, analysis_recovery, analysis_deaths])
analysis_all

,Date,Province,Value,Data
0,2020-03-05,Total RSA,1.0,Cases
1,2020-03-06,Total RSA,1.0,Cases
2,2020-03-07,Total RSA,2.0,Cases
3,2020-03-08,Total RSA,3.0,Cases
4,2020-03-09,Total RSA,7.0,Cases
...,...,...,...,...
1865,2020-09-03,WC,3946.0,Deaths
1866,2020-09-04,WC,3950.0,Deaths
1867,2020-09-05,WC,3960.0,Deaths
1868,2020-09-06,WC,3961.0,Deaths


In [40]:
analysis_states = analysis_all.query(f"Province != 'Total RSA'")
analysis_country = analysis_all.query(f"Province == 'Total RSA'")

In [41]:
template_h = '%{y}'

## Plot analysis for provinces

In [42]:
fig_analysis_prov = px.bar(analysis_states, title='Analysis For Provinces',
             x='Date', y='Value', color='Province', animation_frame='Data',
             barmode='relative', color_discrete_sequence=colour_series)

fig_analysis_prov.update_layout(plot_bgcolor="#FFF",hovermode="x", height=650)
fig_analysis_prov.update_xaxes(linecolor="#BCCCDC")
fig_analysis_prov.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_analysis_prov.update_traces(hovertemplate=template_h)
fig_analysis_prov["layout"].pop("updatemenus") # remove play controls
fig_analysis_prov.show()
#plot_analsysis_prov = plot(fig_analysis_prov, output_type='div', include_plotlyjs=False, auto_play=False)

## Plot analysis for deaths

In [43]:
analysis_states_deaths = analysis_deaths.query(f"Province != 'Total RSA'")

In [44]:
fig_analysis_death = px.bar(analysis_states_deaths, title='Analysis For Deaths',
             x='Date', y='Value', color='Province',
             barmode='relative', color_discrete_sequence=colour_series)

fig_analysis_death.update_layout(plot_bgcolor="#FFF")
fig_analysis_death.update_xaxes(linecolor="#BCCCDC")
fig_analysis_death.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_analysis_death.update_traces(hovertemplate=None)
fig_analysis_death.update_layout(hovermode="x")
fig_analysis_death.show()
#plot_analsysis_deaths = plot(fig_analysis_death, output_type='div', include_plotlyjs=False)

## Plot analysis for South Africa

In [45]:
states_tests['Province'] = 'Total RSA'
states_tests['Data'] = 'Tests'
states_tests = states_tests.rename(columns={'Total RSA':'Value'})

In [46]:
analysis_country = pd.concat([analysis_country, states_tests])
analysis_country

,Date,Province,Value,Data
0,2020-03-05,Total RSA,1,Cases
1,2020-03-06,Total RSA,1,Cases
2,2020-03-07,Total RSA,2,Cases
3,2020-03-08,Total RSA,3,Cases
4,2020-03-09,Total RSA,7,Cases
...,...,...,...,...
182,2020-09-03,Total RSA,3747101,Tests
183,2020-09-04,Total RSA,3765700,Tests
184,2020-09-05,Total RSA,3783823,Tests
185,2020-09-06,Total RSA,3800190,Tests


In [47]:
px_data_sa = px.line(analysis_country, x='Date', y='Value', color='Data', line_shape='spline')
fig_analysis_sa = make_subplots(specs=[[{"secondary_y": True}]])

In [48]:
fig_analysis_sa.add_trace(px_data_sa['data'][0], secondary_y=True)
fig_analysis_sa.add_trace(px_data_sa['data'][1], secondary_y=True)
fig_analysis_sa.add_trace(px_data_sa['data'][2], secondary_y=True)
fig_analysis_sa.add_trace(px_data_sa['data'][3], secondary_y=True)
fig_analysis_sa.add_trace(px_data_sa['data'][4], secondary_y=False)

fig_analysis_sa.update_yaxes(title_text="Rest of Data", secondary_y=True)
fig_analysis_sa.update_yaxes(title_text="Tests", secondary_y=False)
fig_analysis_sa.update_layout(title="Analysis for South Africa")

fig_analysis_sa.update_layout(plot_bgcolor="#FFF",hovermode="x")

fig_analysis_sa.update_xaxes(showspikes=True, spikesnap="cursor", spikemode="across", spikethickness=1, linecolor="#BCCCDC")
fig_analysis_sa.update_yaxes(showspikes=True, spikethickness=1, linecolor="#BCCCDC", gridcolor='#D3D3D3')
fig_analysis_sa.update_layout(spikedistance=1000, hoverdistance=100)

fig_analysis_sa.update_traces(hovertemplate=None)
#plot_analysis_sa = plot(fig_analysis_sa, output_type='div', include_plotlyjs=False)

## Summary

In [49]:
latest_date = caselast.strftime("%d %B %Y")
f_date = caselast.strftime("%Y-%m-%d")

In [50]:
analysis_latest = analysis_country.query(f"Date == '{f_date}'")
analysis_latest

,Date,Province,Value,Data
186,2020-09-07,Total RSA,639362,Cases
186,2020-09-07,Total RSA,57803,Active
186,2020-09-07,Total RSA,566555,Recovered
186,2020-09-07,Total RSA,15004,Deaths
186,2020-09-07,Total RSA,3808 49,Tests


In [51]:
latest_cases = format_comma(analysis_latest.iloc[0]['Value'])
latest_active = analysis_latest.iloc[1]['Value']
latest_recovery = analysis_latest.iloc[2]['Value']
latest_deaths = analysis_latest.iloc[3]['Value']
latest_tests = analysis_latest.iloc[4]['Value']

In [52]:
latest_cases, latest_active, latest_recovery, latest_deaths, latest_tests

('639,362', 57803.0, 566555.0, 15004.0, '3808 49')

## Plot analysis per province

In [53]:
max_states = max(analysis_states['Value']) * 1.05
max_states

223741.35

In [54]:
fig_analaysis_prov2 = px.line(analysis_states, title='Analysis Per Provinces',
             x='Date', y='Value', color='Data', animation_frame='Province',
             line_shape='spline', range_y=[0, max_states],
             color_discrete_sequence=colour_series)

fig_analaysis_prov2.update_layout(plot_bgcolor="#FFF")
fig_analaysis_prov2.update_xaxes(linecolor="#BCCCDC")
fig_analaysis_prov2.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_analaysis_prov2.update_xaxes(showspikes=True, spikesnap="cursor", spikemode="across", spikethickness=1)
fig_analaysis_prov2.update_yaxes(showspikes=True, spikethickness=1)
fig_analaysis_prov2.update_layout(spikedistance=1000, hoverdistance=100)

fig_analaysis_prov2.update_traces(hovertemplate=template_h)
fig_analaysis_prov2.update_layout(hovermode="x")
fig_analaysis_prov2["layout"].pop("updatemenus") # remove play controls

fig_analaysis_prov2.show()
#plot_analsysis_prov2 = plot(fig_analaysis_prov2, output_type='div', include_plotlyjs=False, auto_play=False)

# Daily Analysis For All Data

In [55]:
state_filter_t

['Total RSA', 'EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

In [56]:
def shape_daily(states_df_i, label, fil=True):
    if fil:
        all_df = states_df_i[state_filter_t]
    else:
        all_df = states_df_i
    daily_df_i = all_df.diff()
    daily_df_i = daily_df_i[1:]
    daily_df = daily_df_i.reset_index()
    daily_df = daily_df.rename(columns={'index':'Date'})
    daily_melt_df = daily_df.melt(id_vars='Date', var_name='Province', value_name='Value')
    daily_melt_df['Data'] = label
    return daily_melt_df, daily_df_i

daily_melt_cases, daily_cases = shape_daily(states_cases_i, 'Cases')
daily_melt_active, x = shape_daily(filter_active_i, 'Active')
daily_melt_recovery, x = shape_daily(states_recovery_i, 'Recovery')
daily_melt_deaths, x = shape_daily(states_deaths_i, 'Deaths')

daily_melt_active

,Date,Province,Value,Data
0,2020-03-06,Total RSA,0.0,Active
1,2020-03-07,Total RSA,1.0,Active
2,2020-03-08,Total RSA,1.0,Active
3,2020-03-09,Total RSA,4.0,Active
4,2020-03-10,Total RSA,0.0,Active
...,...,...,...,...
1855,2020-09-03,WC,212.0,Active
1856,2020-09-04,WC,-441.0,Active
1857,2020-09-05,WC,-136.0,Active
1858,2020-09-06,WC,117.0,Active


In [57]:
states_cases_smoothed = daily_cases.rolling(7,
    win_type='gaussian',
    min_periods=1,
    center=True).mean(std=2).round()
states_cases_smoothed

,Total RSA,EC,FS,GP,KZN,LP,MP,NC,NW,WC
2020-03-06,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-07,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-08,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-09,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-10,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-09-03,2012.0,121.0,327.0,487.0,324.0,84.0,126.0,204.0,156.0,194.0
2020-09-04,1917.0,117.0,317.0,447.0,297.0,80.0,112.0,208.0,151.0,194.0
2020-09-05,1814.0,111.0,306.0,409.0,266.0,77.0,108.0,208.0,144.0,188.0
2020-09-06,1626.0,97.0,287.0,355.0,222.0,69.0,99.0,201.0,126.0,169.0


In [58]:
#idx_start = np.searchsorted(smoothed, 10)
#smoothed = smoothed.iloc[idx_start:]

daily_smoothed = states_cases_smoothed.reset_index()
daily_smoothed = daily_smoothed.rename(columns={'index':'Date'})
daily_melt_smoothed = daily_smoothed.melt(id_vars='Date', var_name='Province', value_name='Value')
daily_melt_smoothed['Data'] = 'Cases Smoothed'
daily_melt_smoothed

,Date,Province,Value,Data
0,2020-03-06,Total RSA,1.0,Cases Smoothed
1,2020-03-07,Total RSA,1.0,Cases Smoothed
2,2020-03-08,Total RSA,2.0,Cases Smoothed
3,2020-03-09,Total RSA,2.0,Cases Smoothed
4,2020-03-10,Total RSA,3.0,Cases Smoothed
...,...,...,...,...
1855,2020-09-03,WC,194.0,Cases Smoothed
1856,2020-09-04,WC,194.0,Cases Smoothed
1857,2020-09-05,WC,188.0,Cases Smoothed
1858,2020-09-06,WC,169.0,Cases Smoothed


In [59]:
daily_all = pd.concat([daily_melt_cases, daily_melt_smoothed, daily_melt_active, daily_melt_recovery, daily_melt_deaths])

In [60]:
daily_country = daily_all.query(f"Province == 'Total RSA'")
daily_states = daily_all.query(f"Province != 'Total RSA'")
daily_country

,Date,Province,Value,Data
0,2020-03-06,Total RSA,0.0,Cases
1,2020-03-07,Total RSA,1.0,Cases
2,2020-03-08,Total RSA,1.0,Cases
3,2020-03-09,Total RSA,4.0,Cases
4,2020-03-10,Total RSA,0.0,Cases
...,...,...,...,...
181,2020-09-03,Total RSA,174.0,Deaths
182,2020-09-04,Total RSA,115.0,Deaths
183,2020-09-05,Total RSA,101.0,Deaths
184,2020-09-06,Total RSA,110.0,Deaths


In [64]:
states_tests_i
daily_df_i = states_tests_i.diff()
daily_df_i = daily_df_i[1:]
daily_df_i

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [61]:
daily_melt_tests, x = shape_daily(states_tests_i, 'Tests', False)
daily_melt_tests

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
daily_country = pd.concat([daily_country, daily_melt_tests])
daily_country

## Plot daily change for South Africa

In [ ]:
px_daily_sa = px.line(daily_country, x='Date', y='Value', color='Data') #, line_shape='spline'
fig_daily_sa = make_subplots(rows=1, cols=2, specs=[[{},{"secondary_y": True}]], y_title="Value")

In [ ]:
#visible="legendonly"
fig_daily_sa.add_trace(px_daily_sa['data'][0], row=1, col=1)
fig_daily_sa.add_trace(px_daily_sa['data'][1], row=1, col=1)
fig_daily_sa.add_trace(px_daily_sa['data'][2], secondary_y=False, row=1, col=2)
fig_daily_sa.add_trace(px_daily_sa['data'][3], secondary_y=False, row=1, col=2)
fig_daily_sa.add_trace(px_daily_sa['data'][4], secondary_y=False, row=1, col=2)
fig_daily_sa.add_trace(px_daily_sa['data'][5], secondary_y=True, row=1, col=2)

fig_daily_sa.update_layout(plot_bgcolor="#FFF")
fig_daily_sa.update_xaxes(linecolor="#BCCCDC")
fig_daily_sa.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_daily_sa.update_yaxes(title_text="Tests", secondary_y=True)
#fig_daily_sa.update_yaxes(title_text="Tests", secondary_y=False)
fig_daily_sa.update_layout(title="Daily Change for South Africa")

fig_daily_sa.update_xaxes(showspikes=True, spikesnap="cursor", spikemode="across", spikethickness=1)
fig_daily_sa.update_yaxes(showspikes=True, spikethickness=1, spikemode="across")
fig_daily_sa.update_layout(spikedistance=1000, hoverdistance=100)

fig_daily_sa.update_traces(hovertemplate=None)
fig_daily_sa.update_layout(hovermode="x")

fig_daily_sa.show()
#plot_daily_sa = plot(fig_daily_sa, output_type='div', include_plotlyjs=False)

## Plot daily change for provinces

In [ ]:
max_daily = max(daily_states['Value']) * 1.05
min_daily = min(daily_states['Value']) * 1.05
max_daily, min_daily

In [ ]:
fig_daily_prov = px.line(daily_states, title='Daily Change For Provinces',
             x='Date', y='Value', color='Data', animation_frame='Province',
             range_y=[min_daily, max_daily], line_shape='spline',
             color_discrete_sequence=colour_series)

fig_daily_prov.update_layout(plot_bgcolor="#FFF", height=650)
fig_daily_prov.update_xaxes(linecolor="#BCCCDC")
fig_daily_prov.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_daily_prov.update_traces(hovertemplate=template_h)
fig_daily_prov.update_layout(hovermode="x")
fig_daily_prov["layout"].pop("updatemenus") # remove play controls

fig_daily_prov.update_xaxes(showspikes=True, spikesnap="cursor", spikemode="across", spikethickness=1)
fig_daily_prov.update_yaxes(showspikes=True, spikethickness=1)
fig_daily_prov.update_layout(spikedistance=1000, hoverdistance=100)

fig_daily_prov.show()
#plot_daily_prov = plot(fig_daily_prov, output_type='div', include_plotlyjs=False, auto_play=False)

# Evaluate district data

In [ ]:
districts_gp = {
'date':'date',
'Ekurhuleni\tCases':'Ekurhuleni',
'Johannesburg\tCases':'Johannesburg',
'Sedibeng\tCases':'Sedibeng',
'Tshwane\tCases':'Tshwane',
'West Rand\tCases':'West Rand',
'West Rand\tCases':'West Rand',
'GP Unallocated\tCases':'Unknown'
}
districts = districts_gp.keys()

In [ ]:
file_name = 'provincial_' + 'gp' + '_cumulative.csv'  
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/district_data/' + file_name
states_district = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True).sort_index()

states_district_filter = states_district[districts]
col_tol = states_district_filter.sum(axis=1, numeric_only=True)
pd.options.mode.chained_assignment = None
states_district_filter['Total'] = col_tol

#states = states_filter_wp
states_district_filter.tail()

# Rt Model 1

## Data Import

In [ ]:
#dsfsi
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/calc/calculated_rt_sa_provincial_cumulative.csv'
states_all_rt_i = pd.read_csv(url, parse_dates=['date'], dayfirst=True, squeeze=True, index_col=[0,1])
states_all_rt_i

In [ ]:
states_all_rt = states_all_rt_i.copy()
states_all_rt = states_all_rt.reset_index()
states_all_rt = states_all_rt.rename(columns={'date':'Date'})
states_all_rt = states_all_rt.rename(columns={'ML':'Rt'})
states_all_rt = states_all_rt.rename(columns={'state':'Province'})
states_all_rt

## Setup Rt

In [ ]:
#state_single = states_rt.filter(like='Total RSA', axis=0) # for index data
state_single = states_all_rt.query("Province == 'Total RSA'")
state_single

In [ ]:
X0rt1 = state_single.iloc[1]['Date']
X0rt1

In [ ]:
latest_result_rt = state_single.iloc[-1]
X2rt1 = latest_result_rt['Date']
latest_d_rt1 = X2rt1.strftime("%d %B %Y")
rt1 = latest_result_rt['Rt']
latest_d_rt1, rt1

## Latest Rt summary

In [ ]:
rt1_last_df = states_all_rt_i.groupby(level=0)[['ML']].last()
rt1_last_df

In [ ]:
rt1_states = rt1_last_df['ML'].to_dict()

## Country Rt

In [ ]:
state_single["e_plus"] = state_single['High_90'].sub(state_single['Rt'])
state_single["e_minus"] = state_single['Rt'].sub(state_single['Low_90'])

In [ ]:
fig_rt1 = px.line(state_single, x='Date', y='Rt',
              error_y='e_plus', error_y_minus='e_minus',
              title='Model 1: Rt for Covid-19 in South Africa', line_shape='spline')
fig_rt1.update_traces(hovertemplate=None)
fig_rt1.update_layout(hovermode="x")
fig_rt1['data'][0]['error_y']['color'] = 'lightblue'

fig_rt1.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X0rt1,
    y0=1,
    x1=X2rt1,
    y1=1,
    opacity=0.6,
    line=dict(
        color="Crimson",
        width=2,
        dash='dash'
))

fig_rt1.update_layout(plot_bgcolor="#FFF")
fig_rt1.update_xaxes(linecolor="#BCCCDC")
fig_rt1.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_rt1.show()

## Province Rt

In [ ]:
states_rt = states_all_rt.query("Province != 'Total RSA'")
states_rt

In [ ]:
fig_px = px.line(states_rt, x='Date', y='Rt', color='Province')
fig_len = len(fig_px['data'])

In [ ]:
fig_rt_province = make_subplots(rows=3, cols=3,
                    subplot_titles=state_labels,
                    shared_xaxes=True, shared_yaxes=True)

In [ ]:
r = 0
for p in range(fig_len):
    c = (p % 3) + 1
    if (c == 1):
        r+=1
    fig_rt_province.add_trace(fig_px['data'][p], row=r, col=c)
    
    fig_rt_province.add_shape(
    type="line",
    xref="x{0}".format(p+1),
    yref="y{0}".format(p+1),
    x0=X0rt1,
    y0=1,
    x1=X2rt1,
    y1=1,
    opacity=0.6,
    line=dict(
        color="Crimson",
        width=2,
        dash='dash'
    ))

In [ ]:
fig_rt_province.update_layout(title_text="Rt for Covid-19 in South African Provinces", height=700)
fig_rt_province.update_traces(hovertemplate=None)
fig_rt_province.update_layout(hovermode="x")

fig_rt_province.update_layout(plot_bgcolor="#FFF")
fig_rt_province.update_xaxes(linecolor="#BCCCDC")
fig_rt_province.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

# Rt model 2

In [ ]:
url = 'https://raw.githubusercontent.com/dsfsi/covid19za/master/data/calc/calculated_rt_sa_mcmc.csv'
state_rt_mcmc = pd.read_csv(url, parse_dates=['date'], dayfirst=True, squeeze=True)
state_rt_mcmc = state_rt_mcmc.rename(columns={'date':'Date'})
state_rt_mcmc = state_rt_mcmc.rename(columns={'Median':'Rt'})
state_rt_mcmc

In [ ]:
X0rt2 = state_rt_mcmc.iloc[0,:]['Date']
X0rt2

In [ ]:
latest_rt2 = state_rt_mcmc.iloc[-1]
rt2 = round(latest_rt2['Rt'], 2)
rt2

In [ ]:
X2rt2 = latest_rt2['Date']
latest_d_rt2 = X2rt2.strftime("%d %B %Y")
X2rt2, latest_d_rt2

In [ ]:
state_rt_mcmc["e_plus"] = state_rt_mcmc['High_80'].sub(state_rt_mcmc['Rt'])
state_rt_mcmc["e_minus"] = state_rt_mcmc['Rt'].sub(state_rt_mcmc['Low_80'])

In [ ]:
fig_rt2 = px.line(state_rt_mcmc, x='Date', y='Rt',
              error_y='e_plus', error_y_minus='e_minus',
              title='Model 2: Rt for Covid-19 in South Africa', line_shape='spline')
fig_rt2.update_traces(hovertemplate=None)
fig_rt2.update_layout(hovermode="x")
fig_rt2['data'][0]['error_y']['color'] = 'lightblue'

fig_rt2.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X0rt2,
    y0=1,
    x1=X2rt2,
    y1=1,
    opacity=0.6,
    line=dict(
        color="Crimson",
        width=2,
        dash='dash'
))

fig_rt2.update_layout(plot_bgcolor="#FFF")
fig_rt2.update_xaxes(linecolor="#BCCCDC")
fig_rt2.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_rt2.show()

# Forecasts models
## Herd immunity

Herd immunity – estimating the level required to halt the COVID-19 epidemics in affected countries

Ref: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151357/

In [ ]:
Pc = 1-(1/rt2)
Pc * 100

In [ ]:
immune = country_pop * Pc
immune

## Rt trends model

In [ ]:
cases_series = pd.Series(states_cases_i['Total RSA'].values, index=states_cases_i.index.values, name='Cases')

cases_df = cases_series.to_frame()
cases_df = cases_df.reset_index()
cases_df = cases_df.rename(columns={'index':'Date'})
cases_df

In [ ]:
from datetime import timedelta, date

In [ ]:
f = 60

In [ ]:
f2 = 30

In [ ]:
diff = cases_df['Cases'].diff()

In [ ]:
d = diff.values[-1]
d

In [ ]:
r_scenarios = [1.5, 1.4, 1.3, 1.25, 1.2, 1.15, 1.1, 1.075, 1.05, 1.025, 1.0, 0.975, 0.95, 0.925, 0.9, 0.85, 0.8, 0.7, 0.6, 0.5, 0.25, 0.1]
if (rt2 not in r_scenarios):
    r_scenarios.append(rt2)
    r_scenarios.sort(reverse=True)
r_scenarios

In [ ]:
future_projections = None

for r in r_scenarios:
    projection = cases_df.copy()
    lastd = cases_df['Date'].iloc[-1]
    lastc = cases_df['Cases'].iloc[-1]
    d = diff.values[-1]

    for i in range(f):
        lastd += timedelta(days=1)
        newc = lastc + (d * r)
        d = newc - lastc
        lastc = newc

        calc = pd.DataFrame([[lastd, lastc]], columns=['Date', 'Cases'])
        # TODO: consider concat opertion here for faster processing
        projection = projection.append(calc)
        
    projection['R'] = r
    
    if future_projections is None:
        future_projections = projection
    else:
        future_projections = pd.concat([future_projections, projection])

future_projections

In [ ]:
current_forecast = future_projections.query(f"R == {rt2}")
current_forecast

In [ ]:
last_forecast = current_forecast.iloc[-1]
future_f = math.trunc(last_forecast['Cases'])
future_f

In [ ]:
infected = future_f / country_pop * 100
f'{infected:.1f}%'

In [ ]:
Xdt = date.today()
X0f = current_forecast.iloc[0]['Date']
X1f = Xdt + timedelta(days=f2)
X2f = last_forecast['Date']

In [ ]:
max_forecast = max(current_forecast['Cases']) * 1.05
max_country = country_pop * 1.1
max_future = min(max_forecast, max_country)
max_future

In [ ]:
fig_forecast = px.line(current_forecast, x='Date', y='Cases',
               range_y=[0, max_future],
               title='Covid-19 Cases Forecast for Current Rt')
fig_forecast.update_traces(hovertemplate=None)
fig_forecast.update_layout(hovermode="x")

fig_forecast.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=Xdt,
    y0=0,
    x1=Xdt,
    y1=max_future,
    opacity=0.6,
    line=dict(
        color="Black",
        width=2,
        dash='dashdot'
))

fig_forecast.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=X0f,
    y0=immune,
    x1=X2f,
    y1=immune,
    opacity=0.6,
    line=dict(
        color="Crimson",
        width=2,
        dash='dash'
))

fig_forecast.add_annotation(
            x=X1f,
            y=immune * 1.05,
            text="Herd Immunity",
            showarrow=False
)

fig_forecast.update_layout(plot_bgcolor="#FFF")
fig_forecast.update_xaxes(linecolor="#BCCCDC")
fig_forecast.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_forecast.show()

In [ ]:
increasing_forecast = future_projections.query(f"R > 1")

In [ ]:
fig_forecast1 = px.line(increasing_forecast, x='Date', y='Cases',
               animation_frame='R', height=600, range_y=[0, max_country],
               title='Covid-19 Cases Forecast for Increasing Cases (Rt is bigger than 1)')

fig_forecast1.update_layout(plot_bgcolor="#FFF")
fig_forecast1.update_xaxes(linecolor="#BCCCDC")
fig_forecast1.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_forecast1.update_layout(hovermode="x")
#fig_forecast1["layout"].pop("updatemenus") # to remove play controls

fig_forecast1.show()

In [ ]:
linear_forecast = future_projections.query(f"R == 1")
max_linear = max(linear_forecast['Cases'])

In [ ]:
fig_forecast2 = px.line(linear_forecast, x='Date', y='Cases',
               range_y=[0, max_linear],
               title='Covid-19 Cases Forecast for Increasing Cases (Rt is 1)')
fig_forecast2.update_traces(hovertemplate=None)
fig_forecast2.update_layout(hovermode="x")

fig_forecast2.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=Xdt,
    y0=0,
    x1=Xdt,
    y1=max_linear,
    opacity=0.6,
    line=dict(
        color="Black",
        width=2,
        dash='dashdot'
))

fig_forecast2.update_layout(plot_bgcolor="#FFF")
fig_forecast2.update_xaxes(linecolor="#BCCCDC")
fig_forecast2.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_forecast2.show()

In [ ]:
decline_forecast = future_projections.query(f"R < 1")
max_decline = max(decline_forecast['Cases']) * 1.05

In [ ]:
fig_forecast3 = px.line(decline_forecast, x='Date', y='Cases',
               animation_frame='R', height=600, range_y=[0, max_decline],
               title='Covid-19 Cases Forecast for Decreasing Cases (Rt is less than 1)')
fig_forecast3.update_traces(hovertemplate=None)
fig_forecast3.update_layout(hovermode="x")

fig_forecast3.add_shape(
    type="line",
    xref="x",
    yref="y",
    x0=Xdt,
    y0=0,
    x1=Xdt,
    y1=max_decline,
    opacity=0.6,
    line=dict(
        color="Black",
        width=2,
        dash='dashdot'
))

fig_forecast3.update_layout(plot_bgcolor="#FFF")
fig_forecast3.update_xaxes(linecolor="#BCCCDC")
fig_forecast3.update_yaxes(linecolor="#BCCCDC", gridcolor='#D3D3D3')

fig_forecast3.show()

# Matplotlib result
## Rt plots

In [ ]:
repo = 'dsfsi'
url = 'https://raw.githubusercontent.com/' + repo + '/covid19za/master/data/calc/calculated_rt_sa_provincial_cumulative.csv'
# states_all_rt_i already calcualte, but should calc again
states_all_rt_i = pd.read_csv(url,
                     parse_dates=['date'], dayfirst=True,
                     squeeze=True, index_col=[0,1])
states_all_rt_i

In [ ]:
def plot_rt(result, ax, state_name):
    
    ax.set_title(f"{state_name}")
    
    # Colors
    ABOVE = [1,0,0]
    MIDDLE = [1,1,1]
    BELOW = [0,0,0]
    cmap = ListedColormap(np.r_[
        np.linspace(BELOW,MIDDLE,25),
        np.linspace(MIDDLE,ABOVE,25)
    ])
    color_mapped = lambda y: np.clip(y, .5, 1.5)-.5
    
    index = result['ML'].index.get_level_values('date')
    values = result['ML'].values
    
    # Plot dots and line
    ax.plot(index, values, c='k', zorder=1, alpha=.25)
    ax.scatter(index,
               values,
               s=40,
               lw=.5,
               c=cmap(color_mapped(values)),
               edgecolors='k', zorder=2)
    
    # Aesthetically, extrapolate credible interval by 1 day either side
    lowfn = interp1d(date2num(index),
                     result['Low_90'].values,
                     bounds_error=False,
                     fill_value='extrapolate')
    
    highfn = interp1d(date2num(index),
                      result['High_90'].values,
                      bounds_error=False,
                      fill_value='extrapolate')
    
    extended = pd.date_range(start=pd.Timestamp('2020-03-01'),
                             end=index[-1]+pd.Timedelta(days=1))
    
    ax.fill_between(extended,
                    lowfn(date2num(extended)),
                    highfn(date2num(extended)),
                    color='k',
                    alpha=.1,
                    lw=0,
                    zorder=3)

    ax.axhline(1.0, c='k', lw=1, label='$R_t=1.0$', alpha=.25);
    
    # Formatting
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.DayLocator())
    
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    ax.yaxis.tick_right()
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.margins(0)
    ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
    ax.margins(0)
    ax.set_ylim(0.0, 5.0)
    ax.set_xlim(pd.Timestamp('2020-03-06'), result.index.get_level_values('date')[-1]+pd.Timedelta(days=1))
    #fig.set_facecolor('w')
    
    return ax

In [ ]:
credit = 'Source: covid19trends.co.za - Data: DSFSI'

In [ ]:
country = states_all_rt_i.filter(like='Total RSA', axis=0)
fig_country, ax = plt.subplots(figsize=(600/72,400/72))
ax = plot_rt(country, ax, state_name = '')
ax.set_title(credit, size=12, weight='light')
fig_country.suptitle(f'$R_t$ for COVID-19 in South Africa', size=14)

In [ ]:
states_rt_i = states_all_rt_i.loc[state_filter]

In [ ]:
def all_plot(final_results):
    state_groups = final_results.groupby('state')
    
    ncols = 3
    nrows = int(np.ceil(len(state_groups) / ncols))

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows*3))

    for i, (state_name, result) in enumerate(state_groups):
        axes.flat[i] = plot_rt(result, axes.flat[i], state_name)

    fig.tight_layout()
    fig.set_facecolor('w')
    
    fig.suptitle(f'$R_t$ for COVID-19 in South African Provinces', size=14)
    fig.subplots_adjust(top=0.92)
    
all_plot(states_rt_i)

## Standings plots

In [ ]:
# ZA: South Arica lockdown level data as of 2020/05/03
no_lockdown = [

]
partial_lockdown = [

]
# add items as required -> 'Western Cape','WC',

FULL_COLOR = [.7,.7,.7]
NONE_COLOR = [179/255,35/255,14/255]
PARTIAL_COLOR = [.5,.5,.5]
ERROR_BAR_COLOR = [.3,.3,.3]

In [ ]:
# ZA: df slighty different to US
mr = states_all_rt_i.groupby(level=0)[['ML', 'High_90', 'Low_90']].last()
mr

In [ ]:
def plot_standings(mr, figsize=None, title='Most Recent $R_t$ by Province'):
    if not figsize:
        figsize = ((15.9/50)*len(mr)+.1,2.5)
        
    fig, ax = plt.subplots(figsize=figsize)

    ax.set_title(title)
    err = mr[['Low_90', 'High_90']].sub(mr['ML'], axis=0).abs()
    bars = ax.bar(mr.index,
                  mr['ML'],
                  width=.825,
                  color=FULL_COLOR,
                  ecolor=ERROR_BAR_COLOR,
                  capsize=2,
                  error_kw={'alpha':.5, 'lw':1},
                  yerr=err.values.T)

    for bar, state_name in zip(bars, mr.index):
        if state_name in no_lockdown:
            bar.set_color(NONE_COLOR)
        if state_name in partial_lockdown:
            bar.set_color(PARTIAL_COLOR)

    labels = mr.index.to_series().replace({'District of Columbia':'DC'})
    ax.set_xticklabels(labels, rotation=90, fontsize=11)
    ax.margins(0)
    ax.set_ylim(0,2.)
    ax.axhline(1.0, linestyle=':', color='k', lw=1)

    leg = ax.legend(handles=[
                        Patch(label='Full', color=FULL_COLOR),
                        Patch(label='Partial', color=PARTIAL_COLOR),
                        Patch(label='None', color=NONE_COLOR)
                    ],
                    title='Lockdown',
                    ncol=3,
                    loc='upper left',
                    columnspacing=.75,
                    handletextpad=.5,
                    handlelength=1)

    leg._legend_box.align = "left"
    fig.set_facecolor('w')
    return fig, ax

mr.sort_values('ML', inplace=True)
plot_standings(mr);

In [ ]:
mr.sort_values('High_90', inplace=True)
plot_standings(mr);

In [ ]:
show = mr[mr.High_90.le(1)].sort_values('ML')
fig, ax = plot_standings(show, title='Likely Under Control');

In [ ]:
show = mr[mr.Low_90.ge(1.0)].sort_values('Low_90')
fig, ax = plot_standings(show, title='Likely Not Under Control');
ax.get_legend().remove()